In [1]:
import requests
from pyquery import PyQuery as pq

page_source = requests.get("https://rate.bot.com.tw/xrt?Lang=zh-TW")

doc = pq(page_source.text)('tbody')
#print(doc('tr'))
for currency in doc('tr').items():
    print(currency('.visible-phone').text(), end =',')
    print(currency('.rate-content-cash').eq(1).text())


 美金 (USD),31.165
 港幣 (HKD),3.994
 英鎊 (GBP),40.6
 澳幣 (AUD),23
 加拿大幣 (CAD),23.88
 新加坡幣 (SGD),22.84
 瑞士法郎 (CHF),31.3
 日圓 (JPY),0.2777
 南非幣 (ZAR),-
 瑞典幣 (SEK),3.56
 紐元 (NZD),21.54
 泰幣 (THB),1.0017
 菲國比索 (PHP),0.6468
 印尼幣 (IDR),0.00248
 歐元 (EUR),35.77
 韓元 (KRW),0.02956
 越南盾 (VND),0.00145
 馬來幣 (MYR),7.911
 人民幣 (CNY),4.52


In [36]:
import pandas as pd

history_page_source = requests.get("https://rate.bot.com.tw/xrt/quote/l6m/AUD")
doc2 = pq(history_page_source.text)
AUD_dict = {}
for currency in doc2('tr').items():
    AUD_dict[currency('.text-center').eq(0).text()] = currency('.rate-content-cash').eq(1).text()
    #print(currency('.text-center').eq(0).text(), end = ',')
    #print(currency('.rate-content-cash').eq(1).text())

AUD_df = pd.DataFrame(list(AUD_dict.items()), columns=['Date', 'DateValue'])
AUD_df = AUD_df.drop(AUD_df.index[0])
AUD_df['DateValue'] = AUD_df['DateValue'].astype('float64')

statistical_data = {}

statistical_data['澳幣本日現金賣出匯率'] = AUD_df['DateValue'].iloc[0]
statistical_data['澳幣半年內最高匯率'] = AUD_df['DateValue'].max()
statistical_data['澳幣半年內最低匯率'] = AUD_df['DateValue'].min()
statistical_data['澳幣半年內匯率第一四分位率'] = AUD_df['DateValue'].quantile(.25)
statistical_data['澳幣半年內匯率中位數'] = AUD_df['DateValue'].quantile(.5)
statistical_data['澳幣半年內匯率第三四分位率'] = AUD_df['DateValue'].quantile(.75)
statistical_data['半年內詳細資料'] = 'https://rate.bot.com.tw/xrt/quote/l6m/AUD'

print(statistical_data)



{'澳幣本日現金賣出匯率': 22.99, '澳幣半年內最高匯率': 23.13, '澳幣半年內最低匯率': 22.16, '澳幣半年內匯率第一四分位率': 22.505, '澳幣半年內匯率中位數': 22.81, '澳幣半年內匯率第三四分位率': 22.982499999999998, '半年內詳細資料': 'https://rate.bot.com.tw/xrt/quote/l6m/AUD'}


In [56]:
from email.mime.text import MIMEText
from smtplib import SMTP
import configparser 
import smtplib
import sys

config = configparser.ConfigParser()
config.read('config.ini')

sender = config['email']['account']
passwd = config['email']['passwd']
receivers = config['email']['receiver']

msg = MIMEText("""<tr>
                <td>澳幣本日現金賣出匯率</td>
                <td>{}</td>
                </tr>
                <tr>
                <td>澳幣半年內最高匯率</td>
                <td>{}</td>
                </tr>
                <tr>
                <td>澳幣半年內最低匯率</td>
                <td>{}</td>
                </tr>
                <tr>
                <td>澳幣半年內匯率第一四分位率</td>
                <td>{}</td>
                </tr>
                <tr>
                <td>澳幣半年內匯率中位數</td>
                <td>{}</td>
                </tr>
                <tr>
                <td>澳幣半年內匯率第三四分位率</td>
                <td>{}</td>
                </tr>
                <tr>
                <td>半年內詳細資料</td>
                <td>{}</td>
                </tr>
                """.format(AUD_df['DateValue'].iloc[0], AUD_df['DateValue'].max(), AUD_df['DateValue'].min(), AUD_df['DateValue'].quantile(.25), AUD_df['DateValue'].quantile(.5), AUD_df['DateValue'].quantile(.75), 'https://rate.bot.com.tw/xrt/quote/l6m/AUD') , 'html')
msg['Subject'] = "python email test"
msg['From'] = sender
msg['To'] = receivers
  
smtp = smtplib.SMTP_SSL("smtp.gmail.com", 465)

smtp.ehlo() 
smtp.login(sender, passwd)
 
smtp.send_message(msg)
smtp.close()

